In [ ]:
import sqlite3
import pandas as pd
import os
import sys
from Bio.SubsMat import MatrixInfo as matlist
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
conn = sqlite3.connect('../../mismatch_db.db')

In [ ]:
mismatch = pd.read_sql_query("SELECT * FROM mismatch", conn)
mismatch = mismatch.astype({"exon_start_prim": "Int64", "exon_stop_prim":"Int64", "exon_start_hum": "Int64", "exon_stop_hum":"Int64"})

n_in_genom = pd.read_sql_query("""
SELECT mismatch.*, protein.prot_ID, transcript.sequence_genomic
FROM mismatch
JOIN protein ON mismatch.prot_prim = protein.prot_ID
JOIN transcript ON protein.transcript_ID = transcript.transcript_ID
""", conn)
n_in_genom = n_in_genom.astype({"exon_start_prim": "Int64", "exon_stop_prim":"Int64", "exon_start_hum": "Int64", "exon_stop_hum":"Int64"})

prim_introns = pd.read_sql_query("""
SELECT mismatch_ID, mismatch.prot_prim, exon_intron_map.'type', exon_intron_map.number_elem, exon_intron_map.seq
FROM mismatch
JOIN protein ON mismatch.prot_prim = protein.prot_ID
JOIN transcript ON protein.transcript_ID = transcript.transcript_ID
JOIN exon_intron_map ON transcript.transcript_ID = exon_intron_map.transcript_ID  WHERE exon_intron_map.'type' = 'Intron'
""", conn)

human_introns = pd.read_sql_query("""
SELECT mismatch_ID, mismatch.prot_hum, exon_intron_map.'type', exon_intron_map.number_elem, exon_intron_map.seq
FROM mismatch
JOIN protein ON mismatch.prot_hum = protein.prot_ID
JOIN transcript ON protein.transcript_ID = transcript.transcript_ID
JOIN exon_intron_map ON transcript.transcript_ID = exon_intron_map.transcript_ID  WHERE exon_intron_map.'type' = 'Intron'
""", conn)

In [ ]:
# Flagging des mismatch conservés V
counter = 0
total = 0
for index, row in error_list.iloc[:,:].iterrows():
    
    total+=1
    conserved = 0

    mySeq = fasta2List("../../data/raw/uniprot-sequence/"+row[0][19:-6])
    protein_Mismatch = [val for key, val in mySeq.items() if row[2] in key]

    mismtaching_seq = protein_Mismatch[0][row[3]:row[4]]
    for i, j in mySeq.items():
        myAlign = pairwise2.align.localms(j, mismtaching_seq, 2, 0, -.5, -.1, one_alignment_only=True, score_only=True)
        #print(format_alignment(*myAlign[0]))
        if myAlign/len(mismtaching_seq) > 1.8:
            conserved +=1

    if conserved >= 4:
        #print(str(index)+'\t'.join([str(x) for x in row]))
        counter +=1
print(counter)
print(total)

In [ ]:
# Flagging des erreurs d'alignement V
counter = 0
total = 0
for index, row in error_list.iloc[:,:].iterrows():

    mySeq = fasta2List("../../data/raw/uniprot-sequence/"+row[0][19:-6])
    protein_Mismatch = [val for key, val in mySeq.items() if row[2] in key]
    human_prot_Mismatch = [val for key, val in mySeq.items() if row[0][20:-16] in key]
    if human_prot_Mismatch == []:
        pass
    else:
        total +=1
        mismtaching_seq = protein_Mismatch[0]
        human_seq = human_prot_Mismatch[0][row[5]:row[6]]

        myAlign = pairwise2.align.localms(mismtaching_seq, human_seq, 2, -1, -.5, -.1, one_alignment_only=True, score_only=True)
        if myAlign/len(human_seq) > 1.8:
            #print(str(index)+'\t'.join([str(x) for x in row]))
            counter += 1

print(counter)
print(total)


In [ ]:
# Flagging des seq à repeats proteiq V
counter = 0
total = 0
for index, row in error_list.iloc[:,:].iterrows():

    mySeq = fasta2List("../../data/raw/uniprot-sequence/"+row[0][19:-6])
    protein_Mismatch = [val for key, val in mySeq.items() if row[2] in key]
    human_prot_Mismatch = [val for key, val in mySeq.items() if row[0][20:-16] in key]
    if human_prot_Mismatch == []:
        pass
    else:
        total+=1
        mismtaching_seq = protein_Mismatch[0]
        human_seq = human_prot_Mismatch[0][row[5]:row[6]]

        begin_seq = human_prot_Mismatch[0][:row[5]+1]
        end_seq = human_prot_Mismatch[0][row[6]+2:]

        human_prot_Mismatch = begin_seq + end_seq
        myAlign = pairwise2.align.localms(human_prot_Mismatch, human_seq, 2, -1, -.5, -.1, one_alignment_only=True, score_only=True)
        if myAlign/len(human_seq) > 1.8:
            counter += 1

print(counter)
print(total)

In [ ]:
#Flagging des seq à N V


counter = 0
total = 0
for index, row in error_list.iloc[:,:].iterrows():
    my_CDS = []
    subset_exon_pos = myExonicMap.loc[(myExonicMap[0]==row[0]) & (myExonicMap[2]==row[2]) & \
                      (myExonicMap[3]== str(row[3])) & (myExonicMap[4]== str(row[4]))]
    subset_exon = exon_file.loc[exon_file[0]== row[2]]
    subset_intron = intron_file.loc[intron_file[0]== row[2]]
    try:
        for n in range(int(subset_exon_pos[5]), int(subset_exon_pos[6])+1):
            my_exon = subset_exon.loc[subset_exon[3] == n]
            my_intron = subset_intron.loc[subset_intron[3] == n]
            try:
                my_CDS.append(my_exon.iloc[0,6])
                my_CDS.append(my_intron.iloc[0,6])
            except:
                pass
        
        if "N" in ''.join(my_CDS):
            counter +=1
        total +=1
    except:
        pass

print(counter)
print(total)

In [ ]:
# Flagging des 1 exon human vs multiple primate V
primate_exon_pos = pd.read_csv("../../data/mismatch-analysis2/mismatch_exon_pos.tab", sep="\t")
human_exon_pos = pd.read_csv("../../data/mismatch-flagging/human_mismatch_exon_pos.tab", sep="\t")

counter = 0
total = 0
for index, row in human_exon_pos.iloc[:, :].iterrows():
    subset = primate_exon_pos.loc[primate_exon_pos["Alignement"] == row[0]]
    if subset.empty or row[7] == "ERROR":
        continue
    subset = subset.loc[subset["UniprotID"] == row[2]]
    subset = subset.loc[subset["PosStartError"] == row[3]]
    subset = subset.loc[subset["PosStopError"] == row[4]]
    try:
        row_primate = subset.iloc[0,:].to_list()
    except:
        continue
    total+=1
    if (int(row[7])-int(row[8]) == 0) and (row_primate[6]-row_primate[5] >= 2):
        #print(str(index)+'\t'.join([str(x) for x in row]))
        counter += 1

print(counter)
print(total)

In [ ]:
# Stat sur les sites introniques Humains

canonical = 0
non_canonical = 0
human_intron_seq = pd.read_csv("../../data/mismatch-flagging/human_mismatch_intron_seq.tab", sep="\t", header=None)
for index, row in human_intron_seq.iloc[:, :].iterrows():
    if row[6][:2] == "GT" and row[6][-2:] == "AG":
        canonical += 1
    #elif row[6][:2] == "GC" and row[6][-2:] == "AG":
    #    canonical += 1
    #elif row[6][:2] == "AT" and row[6][-2:] == "AC":
    #    canonical += 1
    else:
        non_canonical += 1
print("Canonical (GT/AG, GC/AG, AT/AC) " + str(canonical/(canonical+non_canonical)*100))
print("Non canonical " + str(non_canonical/(canonical+non_canonical)*100))

In [ ]:
# Flagging des mismatch ayant un site humain de splicing non canonique V

counter = 0
total = 0
for index, row in human_exon_pos.iloc[:, :].iterrows():
    subset_intron = human_intron_map.loc[human_intron_map[0] == row[0][20:-15]]
    if subset_intron.empty or row[7] == "ERROR":
        continue
    total += 1
    for i in range(int(row[7]),int(row[8])+2):
        intron = subset_intron.loc[subset_intron[3] == (i-1)]
        if intron.empty:
            continue
        intron = intron.iloc[0,:].to_list()
        #if  (intron[6][:2] == "GT" and intron[6][-2:] == "AG") \
        #    or (intron[6][:2] == "GC" and intron[6][-2:] == "AG") \
        #    or (intron[6][:2] == "AT" and intron[6][-2:] == "AC") and (len(intron[6]) > 30):
        if  (intron[6][:2] == "GT" and intron[6][-2:] == "AG") and (len(intron[6]) > 30):
            pass
        else:
            counter +=1
            #print(str(index)+'\t'.join([str(x) for x in row]))
            break        

print(counter)
print(total)

In [ ]:
# Flagging mismatch with too small introns V
counter = 0
total = 0

for index, row in primate_exon_pos.iloc[:, :].iterrows():
    subset_intron = primate_intron_seq.loc[primate_intron_seq[0] == row[2]]
    if subset_intron.empty or row[5] == "ERROR":
        error+=1
        continue
    total += 1
    for i in range(int(row[5]),int(row[6])):
        intron = subset_intron.loc[subset_intron[3] == i]
        intron = intron.iloc[0,:].to_list()
        if len(intron[6]) <= 29:
            counter +=1
            #print(str(index)+'\t'.join([str(x) for x in row]))
            break

print(counter)
print(total)

In [ ]:
"""
Flagging isoform

import requests
import pandas as pd
import sys

URL = "https://www.uniprot.org/uploadlists/"

params = {
    "from": "ACC+ID",
    "to": "ENSEMBL_ID",
    "format": "tab",
    "query": "Q6H8Q1"
}

r = requests.post(URL, data=params)
print(r.text)

params = {
    "from": "ENSEMBL_TRS_ID",
    "to": "ACC",
    "format": "tab",
    "query": 'ENST00000493231 ENST00000465414 ENST00000476906'
}

r = requests.post(URL, data=params)
print(r.text)
"""